In [1]:
!pip3 install dspy-ai python-dotenv > /dev/null


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 KB 1.9 MB/s eta 0:00:001.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 KB 5.3 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 KB 4.1 MB/s eta 0:00:005.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 KB 4.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 KB 4.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 KB 5.7 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 KB 4.7 MB/s eta 0:00:005.1 MB/s eta 0:00:01
  Using cached requests-2.31.0-py3-no

In [5]:
# Load the ENV variables
import dotenv
import os

dotenv.load_dotenv(".env")


True

In [13]:
# Initialising the models
import dspy

GEN_LM = dspy.OpenAI(model='gpt-3.5-turbo',api_key=os.environ.get("OPENAI_API_KEY"))
EVAL_LM = dspy.OpenAI(model='gpt-4',api_key=os.environ.get("OPENAI_API_KEY"))

dspy.settings.configure(lm=GEN_LM)

In [9]:
# Sample Context 

Context = """Kubernetes progressively rolls out changes to your application or its configuration, while monitoring application health to ensure it doesn't kill all your instances at the same time
          If something goes wrong, Kubernetes will rollback the change for you. Take advantage of a growing ecosystem of deployment solutions
          Restarts containers that fail, replaces and reschedules containers when nodes die, kills containers that don't respond to your user-defined health check, and doesn't advertise them to clients until they are ready to serve.
          Deploy and update Secrets and application configuration without rebuilding your image and without exposing Secrets in your stack configuration.
          Scale your application up and down with a simple command, with a UI, or automatically based on CPU usage."""

Sample_Question = "Does Kubernetes support Horizontal scaling?"
Good_Answer = "Yes, It does. It can be done with simple command and can be automated depending on cpu usage"
Bad_Answer = "No, It only supports vertical scaling. It is complex to scale out"


In [25]:
class EvalSignature(dspy.Signature):
    Context = dspy.InputField(desc="The context for answering the question")
    Question = dspy.InputField(desc="The question to be answered")
    Answer = dspy.InputField(desc="Generated answer to the question")
    Evaluation = dspy.OutputField(desc="Rating between 1 and 5.")

def eval_answer(question,answer):
    print(f"Requested Question: {question}")
    print(f"Generated Answer: {answer}")

    # Parameters for evaluation
    detail = "Is the Answer detail?"
    faithful = "Is the generated answer grounded in the context? Say no if it includes significant facts not in the context."
    overall =   f"Please rate how well this answer answers the question, {question} based on on the context.\n {Context}"

    with dspy.context(lm=EVAL_LM):
        detail = dspy.ChainOfThought(EvalSignature)(Context="N/A",Question=detail,Answer=answer)
        faithful = dspy.ChainOfThought(EvalSignature)(Context=Context,Question=faithful,Answer=answer)
        overall = dspy.ChainOfThought(EvalSignature)(Context=Context,Question=overall,Answer=answer)

    total = float(detail.Evaluation) + float(faithful.Evaluation)*2 + float(overall.Evaluation)

    print(f"Detail   Score: {detail.Evaluation}")
    print(f"Faithful Score: {faithful.Evaluation}")
    print(f"Overall  Score: {overall.Evaluation}")

    return total/5.0


In [26]:
print(eval_answer(question=Sample_Question,answer=Good_Answer))

Requested Question: Does Kubernetes support Horizontal scaling?
Generated Answer: Yes, It does. It can be done with simple command and can be automated depending on cpu usage
Detail   Score: 1
Faithful Score: 5
Overall  Score: 5
3.2


In [27]:
print(eval_answer(question=Sample_Question,answer=Bad_Answer))

Requested Question: Does Kubernetes support Horizontal scaling?
Generated Answer: No, It only supports vertical scaling. It is complex to scale out
Detail   Score: 2
Faithful Score: 1
Overall  Score: 1
1.0
